# Get all the Gartenlaube Text in Wikisource

Mehr als 15.000 Artikel der Gartenlaube sind Ende 2021 in Wikisource erschlossen, ein vielfaches an Seiten der Illustrierten stehen hinter diesen Artikeln. 

## Frage 

Wie kommt man zum Texttranskript aller Gartenlaube-Artikel in Wikisource? (vgl. https://de.wikisource.org/wiki/Benutzer_Diskussion:Jeb#Gartenlaube_runterladen)

## Let's hack

Die MediaWiki-API bietet eine Vielzahl an Methoden um Daten aus MediaWikis - wie Wikisource - maschinell und im großen Stil zu extrahieren. Für große Projekte wie die Gartenlaube, die letztlich auch so sturkturiert sind wie sie strukturiert sind, geht es aber nicht ganz ohne Code. 

Was muss vorab beachtet werden?

* Der Text von Wikisource-Artikeln liegt nicht in den Seiten der strukturierten Zeitschriftenartikel, sondern in den Wiki-Artikel des `Seite:`-Namespaces
* Ein Wikisource-Großprojekt wie Die Gartenlaube ist nicht in einer einzigen Projektkategorie organisiert. Die einzelnen Seiten liegen in Jahrgangskategorien vor, die selbst eine Unterkategorie der Gartenlaube-Kategorie sind.
* Die MediaWiki-API bietet eine Extension an, um möglichst "Plain"-Text zu erhalten. Diese Extension `TextExctracts`(https://www.mediawiki.org/wiki/Extension:TextExtracts/de) ist aber für die Wikisource nicht verfügbar, da hier die Extension Proofread dies technisch gegenwärtig nicht ermöglicht. Daher ist ein Text-Output nur in einem gerenderte HTML oder im Wikitext möglich. Beide Varianten werden am Ende dieses Skripts im Output vereint.
* Mit der Python-Library `mwparserfromhell` wird noch zusätzlich ein plaintext ausgegeben, dieser Text basiert auf dem Wikicode, verliert aber sämtliche Textinhalte, die in Vorlagen gespeichert waren. Konkret sind dies Überschriften und Bildunterschriften. Dieser Volltext ist daher nur als Referenzwert zu verstehen und mit Vorsicht zu genießen.

### 1. Schritt - Alle Jahrgangskategorien parsen

In [29]:
import requests

S = requests.Session()

URL = "https://de.wikisource.org/w/api.php"

PARAMS = {
    "action": "query",
    "cmtitle": "Kategorie:Die Gartenlaube",
    "cmlimit":500,
    "cmtype": "subcat",
    "list": "categorymembers",
    "format": "json"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

Gartenlaube_SubCat = DATA["query"]["categorymembers"]

#for SubCat in Gartenlaube_SubCat:
#    print(SubCat["title"])

### 2. Für jede Subkategorie der Gartenlaube, Seiten des "Seite:"-Namespaces abrufen

In [48]:

pages = []
retDict = {}
for SubCat in Gartenlaube_SubCat:
    catPages = []
    PARAMS = {
    "action": "query",
    "gcmtitle": SubCat["title"],
    "gcmlimit": 500,
    "gcmnamespace":102,
    "generator": "categorymembers",
    "prop": "proofread",
    "format": "json"
    }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    
    try:
        Gartenlaube_Seiten = DATA["query"]["pages"]
        #print(DATA)

        for Pages in Gartenlaube_Seiten:
            print(Gartenlaube_Seiten[Pages])
            pagedet = {}
            pagedet["pageid"] = Gartenlaube_Seiten[Pages]["pageid"]
            pagedet["title"] = Gartenlaube_Seiten[Pages]["title"]
            pagedet["proofread"] = Gartenlaube_Seiten[Pages]["proofread"]
            catPages.append(pagedet)

        try:
            PARAMS["gcmcontinue"] = DATA["continue"]["gcmcontinue"]
            R = S.get(url=URL, params=PARAMS)
            DATA = R.json()
            Gartenlaube_Seiten = DATA["query"]["pages"]
            for Pages in Gartenlaube_Seiten:
                pagedet = {}
                pagedet["pageid"] = Gartenlaube_Seiten[Pages]["pageid"]
                pagedet["title"] = Gartenlaube_Seiten[Pages]["title"]
                pagedet["proofread"] = Gartenlaube_Seiten[Pages]["proofread"]
                catPages.append(pagedet)
                #print(Pages["title"])
        except KeyError:
            pass

        
    except KeyError:
        pass
    
    retDict[SubCat["title"]]=catPages
pages.append(retDict)
#print(pages)


{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f53030990040f0b083c4836011d01dcc3dc18|197747', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'180682': {'pageid': 180682, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1853) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '197740': {'pageid': 197740, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1853) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '197741': {'pageid': 197741, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1853) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '197742': {'pageid': 197742, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1853) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '197743': {'pageid': 197743, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1853) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '197744': {'pageid': 197744, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1853) 006.jp

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f53070990040f0b083c4836011d01dcc3dc18|351335', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'351541': {'pageid': 351541, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1857) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '351174': {'pageid': 351174, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1857) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '351098': {'pageid': 351098, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1857) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '397109': {'pageid': 397109, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1857) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '397111': {'pageid': 397111, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1857) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '351507': {'pageid': 351507, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1857) 006.jp

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f530c0990040f0b083c4836011d01dcc3dc18|353193', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'353287': {'pageid': 353287, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1862) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '353656': {'pageid': 353656, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1862) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '353342': {'pageid': 353342, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1862) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '353072': {'pageid': 353072, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1862) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '353549': {'pageid': 353549, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1862) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '353398': {'pageid': 353398, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1862) 006.jp

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f53100990040f0b083c4836011d01dcc3dc18|265407', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'267966': {'pageid': 267966, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1866) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '267967': {'pageid': 267967, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1866) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '267969': {'pageid': 267969, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1866) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '267973': {'pageid': 267973, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1866) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '265641': {'pageid': 265641, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1866) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '267983': {'pageid': 267983, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1866) 006.jp

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f53140990040f0b083c4836011d01dcc3dc18|250099', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'250093': {'pageid': 250093, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1870) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '250094': {'pageid': 250094, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1870) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '250095': {'pageid': 250095, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1870) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '248514': {'pageid': 248514, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1870) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '248569': {'pageid': 248569, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1870) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '248562': {'pageid': 248562, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1870) 006.jp

...

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f531d0990040f0b083c4836011d01dcc3dc18|162081', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'162073': {'pageid': 162073, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1879) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '162074': {'pageid': 162074, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1879) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '162075': {'pageid': 162075, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1879) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '162076': {'pageid': 162076, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1879) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '162077': {'pageid': 162077, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1879) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '162078': {'pageid': 162078, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1879) 006.jp

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f53210990040f0b083c4836011d01dcc3dc18|122359', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'122351': {'pageid': 122351, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1883) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '122352': {'pageid': 122352, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1883) 002.jpg', 'proofread': {'quality': 3, 'quality_text': 'Korrigiert'}}, '122353': {'pageid': 122353, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1883) 003.jpg', 'proofread': {'quality': 3, 'quality_text': 'Korrigiert'}}, '122354': {'pageid': 122354, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1883) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '122355': {'pageid': 122355, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1883) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '122356': {'pageid': 122356, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1883

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f53260990040f0b083c4836011d01dcc3dc18|150328', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'150320': {'pageid': 150320, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1888) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '150321': {'pageid': 150321, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1888) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '150322': {'pageid': 150322, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1888) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '150323': {'pageid': 150323, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1888) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '150324': {'pageid': 150324, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1888) 005.jpg', 'proofread': {'quality': 0, 'quality_text': 'Sofort fertig'}}, '150325': {'pageid': 150325, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1888)

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f532b0990040f0b083c4836011d01dcc3dc18|251204', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'251197': {'pageid': 251197, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1893) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '251198': {'pageid': 251198, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1893) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '251199': {'pageid': 251199, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1893) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '251064': {'pageid': 251064, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1893) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '251066': {'pageid': 251066, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1893) 005.jpg', 'proofread': {'quality': 0, 'quality_text': 'Sofort fertig'}}, '251201': {'pageid': 251201, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1893)

{'batchcomplete': '', 'continue': {'gcmcontinue': 'page|303a3204362a4c503244402a522c3204098e0f532f0990040f0b083c4836011d01dcc3dc18|142313', 'continue': 'gcmcontinue||'}, 'query': {'pages': {'142305': {'pageid': 142305, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1897) 001.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '142306': {'pageid': 142306, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1897) 002.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '142307': {'pageid': 142307, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1897) 003.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '142308': {'pageid': 142308, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1897) 004.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '142309': {'pageid': 142309, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1897) 005.jpg', 'proofread': {'quality': 4, 'quality_text': 'Fertig'}}, '142310': {'pageid': 142310, 'ns': 102, 'title': 'Seite:Die Gartenlaube (1897) 006.jp

### 3. Für jede Seite die zugehörigen Text auslesen

* Für jeden Jahrgang wird ein JSON-File erzeugt nach dem Muster `GartenlaubeSeitenText_{Jahrgangskategorie}_{Timestamp}.json`und beinhaltet ein JSON-Objekt mit folgender Struktur:
```json
 [{"pageid" : {PAGEID},
   "title"   : {PAGETITLE},
   "html"    : {HTML_OUTPUT},
   "wikitext": {WIKI_MARKUP},
   "plaintxt": {mwparserfromhell(WIKI_MARKUP).strip_code)}
   }]
 ```

In [17]:
import mwparserfromhell
import time
import json

#print(pages[0]["Kategorie:Die Gartenlaube (1853)"])
#https://de.wikisource.org/w/api.php?action=parse&format=json&pageid=197745&prop=text%7Cwikitext

for page in pages[0]:
    SeitenText = []
    for wikisourcePage in pages[0][page]:
        print(wikisourcePage)
    
        PARAMS = {
        "action": "parse",
        "pageid": wikisourcePage["pageid"],
        "pageid": 197745,
        "prop":"text|wikitext",
        "format": "json"
        }
        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        SeitenTextDet = {}
        SeitenTextDet["pageid"] = wikisourcePage["pageid"]
        SeitenTextDet["title"] = wikisourcePage["title"]
        SeitenTextDet["html"] = DATA["parse"]["text"]["*"]
        SeitenTextDet["wikitext"] = DATA["parse"]["wikitext"]["*"]
        wikicode = mwparserfromhell.parse(SeitenTextDet["wikitext"] )
        SeitenTextDet["plaintxt"] = wikicode.strip_code()
        SeitenText.append(SeitenTextDet)

    f = open("output/GartenlaubeSeitenText_"+page+"_"+str(time.time())[0:10]+".json", "w")
    print(json.dumps(SeitenText),file=f)
    f.close()      
        

IndexError: list index out of range

### 4. Merge der Jahrgangs-Files

* Die bestehenden Jahrgangs-Jsonfiles werden hier zu einer Gesamtdatei gemerged. 
* **Achtung**: Dieser Schritt läuft nur als python-Skript via Shell, nicht im Notebook.

In [ ]:
import glob
import time

read_files = glob.glob("/home/paws/Gartenlaube/output/*.json")
print(read_files)

with open("/home/paws/Gartenlaube/output/GartenlaubeSeiten_Text_"+str(time.time())[0:10]+".json", "wb") as outfile:
    outfile.write(b",".join([open(f, "rb").read() for f in read_files]))
